In [1]:
import glob
import os
import librosa
import numpy as np
import ntpath

from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, LSTM, Flatten, InputLayer, Conv1D, MaxPooling1D
from keras.layers import Dropout, BatchNormalization, Bidirectional
from sklearn.model_selection import train_test_split  
from tensorflow.keras.utils import plot_model,to_categorical
import time

import pandas as pd  
import matplotlib.pyplot as plt
from keras.callbacks import EarlyStopping, ModelCheckpoint

import random as rn
import tensorflow as tf

In [16]:
def extract_feature(file_name):
    X, sample_rate = librosa.load(file_name)
    X = X[0:5645]
    stft = np.abs(librosa.stft(X))  
    mfcc = librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T
    chroma = librosa.feature.chroma_stft(S=stft, sr=sample_rate).T  
    mel = librosa.feature.melspectrogram(X, sr=sample_rate).T 
    contrast = librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T  
    tonnetz = librosa.feature.tonnetz(y=librosa.effects.harmonic(X), sr=sample_rate).T  
    return mfcc, chroma, mel, contrast, tonnetz

In [17]:
def parse_audio_files(filename):
    features = []
    try:
        print('process..', filename)
        extracted = extract_feature(filename)
    except Exception as e:
        print('cannot open', e)
    
    ext_features = np.hstack(extracted)
    features.append(ext_features) 
    return np.array(features)

In [18]:
def create_model():  
    model = Sequential()
    model.add(BatchNormalization(axis=-1, input_shape=(12,193))) 
    model.add(LSTM(64, return_sequences=(True)))
    model.add(LSTM(128, return_sequences=True))
    model.add(LSTM(256))
    model.add(Flatten())
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid'))
              
    # model compilation
    model.compile(loss='binary_crossentropy', metrics=['accuracy'])  
    return model

In [19]:
filename = 'C:/Users/ARDI/Downloads/Music/Standard 7 - 1_ Abdul Hadi.mp3';
X = parse_audio_files(filename);

model = create_model()
model.load_weights('./model.h5')

process.. C:/Users/ARDI/Downloads/Music/Standard 7 - 1_ Abdul Hadi.mp3


C:\Users\ARDI\anaconda3\lib\site-packages\librosa\core\spectrum.py:222: UserWarning: n_fft=1024 is too small for input signal of length=706
  warnings.warn(
C:\Users\ARDI\anaconda3\lib\site-packages\librosa\core\spectrum.py:222: UserWarning: n_fft=1024 is too small for input signal of length=353
  warnings.warn(
C:\Users\ARDI\anaconda3\lib\site-packages\librosa\core\spectrum.py:222: UserWarning: n_fft=1024 is too small for input signal of length=177
  warnings.warn(
C:\Users\ARDI\anaconda3\lib\site-packages\librosa\core\spectrum.py:222: UserWarning: n_fft=1024 is too small for input signal of length=89
  warnings.warn(


In [20]:
classes = model.predict(X)
print(classes)
selisih_to_0 = abs(0-classes[0][0])
selisih_to_1 = abs(1-classes[0][0])
if(min(selisih_to_0, selisih_to_1) == selisih_to_0):
    print("negatif")
else:
    print("positif")

[[0.04056299]]
negatif
